In [1]:
import glob
from os import path
import numpy as np
import matplotlib.pyplot as plt
from Modelo.beans import RasterData, SerialFileData

In [188]:
#cubo_ecmwf_e = RasterData.RasterFile(file_full_path=r"D:\ClimatcDataECMWF_ERA5LAND\ECMWF_ERA5LAND\3-tp\2008-2020_tp_D.tif")
#cubo_ecmwf_e = RasterData.RasterFile(file_full_path=r"D:\1-DadosClimaticos\Xavier\prec_daily20100101_20151231.tif")
cubo_ecmwf_e = RasterData.RasterFile(file_full_path=r"D:\1-DadosClimaticos\Xavier\ETo_daily_UT_Brazil_v2_20140101_20170731_s1.tif")

In [189]:
ri = cubo_ecmwf_e.getRasterInformation()
print(ri)
px_width = (float(ri['xmax'])-float(ri['xmin']))/float(ri['nx'])
px_height = +((float(ri['ymin'])-float(ri['ymax']))/float(ri['ny']))
print(px_width)
print(px_height)

{'nx': '168', 'ny': '162', 'xmin': '-74.2500000', 'ymin': '6.2500000', 'xmax': '-32.2500000', 'ymax': '-34.2500000', 'NoData': '9.96920996838686905e+36'}
0.25
0.25


In [1]:
import pandas as pd

coordenadas = pd.read_csv(r"D:\1-DadosClimaticos\Xavier\weather2_coordenadas_todas.csv", sep=";")

coordenadas.head()

,latitude,longitude
0,2.82,-60.66
1,1.83,-61.12
2,0.61,-69.18
3,-0.05,-51.11
4,-0.11,-67.00


In [190]:
import math

def coordenada2pos(coordenada, maximo, w):
    pos = math.floor(((float(coordenada) - float(maximo)) / float(w)))
    return pos*-1 if pos < 0 else pos

print(float(ri['xmax']))
print(float(ri['ymax']) + px_height)

print(coordenada2pos(float(ri['xmax']) + px_width, ri['xmax'], px_width))
print(coordenada2pos(float(ri['ymax']) + px_height, ri['ymax'], px_height))

print("####################")

coor_teste = 5

print(coordenadas['longitude'][coor_teste])
print(coordenadas['latitude'][coor_teste])


print(coordenada2pos(coordenadas['longitude'][coor_teste], ri['xmin'], px_width))
print(coordenada2pos(coordenadas['latitude'][coor_teste], ri['ymin'], px_height))

-32.25
-34.0
1
1
####################
-62.91
-0.96
45
29


In [191]:
coordenadas['pos_x'] = [(lambda c: coordenada2pos(c, ri['xmin'], px_width))(c) for c in coordenadas['longitude']]
coordenadas['pos_y'] = [(lambda c: coordenada2pos(c, ri['ymin'], px_width))(c) for c in coordenadas['latitude']]

In [192]:
cubo_ecmwf_e_ = cubo_ecmwf_e.loadRasterData(isCube=True)

imagem lida, numero de bandas: 1308


In [193]:
stations_stracted_variables = []


def getXY(img, row):
    try:
        return img[row['pos_x']][row['pos_y']] #* 100
    except Exception as e:
        return None
    

for row in coordenadas.to_dict(orient="records"):
    extracted_values = [(lambda c: getXY(c, row))(c) for c in cubo_ecmwf_e_]
    stations_stracted_variables.append(extracted_values)

In [196]:
n_stations = len(stations_stracted_variables)
teste_rotate = np.transpose(stations_stracted_variables)

#df_extracted = pd.DataFrame(teste_rotate, columns=range(1,n_stations+1), index = pd.date_range("2008-01-01","2020-11-01"))
df_extracted = pd.DataFrame(teste_rotate, columns=range(1,n_stations+1), index = pd.date_range("2014-01-01","2017-07-31"))
df_extracted.head()
#df_extracted.to_csv(r"D:\1-DadosClimaticos\ecmwf\ECMWF_TotalPrecipitation_wStations_2008-2020_D.csv", sep=";")
df_extracted.to_csv(r"D:\1-DadosClimaticos\Xavier\Xavier_ETo_wStations_2014-2017_D.csv", sep=";")
